In [1]:
import os,sys,shutil
import numpy as np
import matplotlib.pyplot as plt
import time
from mpm_rect import mpm_rect
from mpm_circ import mpm_circ
sys.path.append(os.getcwd()+'/build')
import MPM_2D
import math
from tqdm import tqdm, trange

In [2]:
#Defining the grid
x_0 = 0; y_0 = 0;                               #Origin of X and Y Axis
lx = 6; ly = 8;                                 #Length of grid in X and Y direction
nx = 18; ny = 24;                               #Number of cells in X and Y direction

Grid = MPM_2D.MPM_Grid(L=[lx, ly], N=[nx, ny], O=[x_0, y_0])
NoNodes = Grid.NoNodes

In [3]:
#Defining the water body
MPV =  mpm_rect([0,0], [2,4], [60,80], 2)
visc = 220.0; density = 997.5; bulk_mod = 2e6;
Water = MPM_2D.MPM_Fluid(visc, density, bulk_mod, MPV)

Water.MPC()[:,18] = -9.81        #Gravity

In [6]:
#Prepare postprocessing
plotstarget_directory = os.path.join(os.getcwd(), "closed_dam_plots")

#Create a new directory to store the plots (delete when there)
if os.path.exists(plotstarget_directory) and os.path.isdir(plotstarget_directory):
    shutil.rmtree(plotstarget_directory)
os.mkdir(plotstarget_directory)
print("Plots will be saved to : ", plotstarget_directory)

def save_plot(current_time):
    dx = lx/nx; dy = ly/ny
    fig, ax = plt.subplots(1, 1, figsize = (5, 5))                                  #creating axis
    ax.set_xlim(x_0,x_0+lx); ax.set_ylim(y_0,y_0+ly)                                #set limits
    #for i in range (nx+1):
         #ax.plot([x_0+dx*i,x_0+dx*i],[y_0,ly+y_0], c=(0.9, 0.9, 0.9, 1.0) )          #Plot X lines
         #for j in range (ny+1):
             #ax.plot([x_0,lx+x_0],[y_0+dy*j,y_0+dy*j], c=(0.9, 0.9, 0.9, 1.0) )      #Plot Y lines
    ax.scatter(Water.MPC()[:,0], Water.MPC()[:,1] ,s=Water.MPC()[:,4]*1e3,c='b')
    ax.set_aspect(aspect='equal')
    filename = 'Water_'+str(current_time)+'.png'
    fig.savefig(os.path.join(plotstarget_directory, filename), dpi=400, bbox_inches="tight")
    plt.close()                                                                     #prevent plotting in jupyter notebook
    return

Plots will be saved to :  /home/karthik/cpp/mpm_2d_cpp/closed_dam_plots


In [7]:
#Running through time steps
t = 0                                               #start time
T = 4; dt = 1e-5                                  #Total time and time step
NoS = int(T/dt)                                     #Number of time steps
noMicroSteps = int(1000)                    #Incremental time step for solver
noMacroSteps = int(NoS/noMicroSteps)        #Number of vtk files

FluidSolver = MPM_2D.MPM_Solver()
FluidSolver.addBC(lambda X: True if (X[1] <= 0.0) else False, vy=0.0)
FluidSolver.addBC(lambda X: True if (X[0] <= 0.0) else False, vx=0.0)
FluidSolver.addBC(lambda X: True if (X[0] >= 6.0) else False, vx=0.0)
alpha = 0

start_time = time.time()
for step in trange (noMacroSteps):
    FluidSolver.Solve(Grid, [Water], noMicroSteps, dt, alpha)
    save_plot(step)
print("--- %s seconds ---" % (time.time() - start_time))

100%|██████████| 399/399 [45:57<00:00,  6.91s/it]--- 2757.8579552173615 seconds ---

